# Automatic Workflow Selection Deep Dive

> Master the WorkflowSelector decision logic for optimal workflow configuration

**25 minutes** | **Level: Advanced**

---

## What You'll Learn

By the end of this notebook, you will be able to:

- Understand the `WorkflowSelector` decision algorithm
- Use `auto_select_workflow()` for automatic configuration
- Interpret `DatasetSizeTier` and `MemoryTier` classifications
- Query memory availability with `get_total_available_memory_gb()` and `get_memory_tier()`
- Predict which workflow tier will be selected for your dataset

---

## Learning Path

**You are here:** Workflow System > **Auto Selection**

```
fit() Quickstart --> Workflow Tiers --> Optimization Goals --> [You are here: Auto Selection]
```

**Recommended flow:**
- **Previous:** [02_workflow_tiers.ipynb](02_workflow_tiers.ipynb) - Understanding tiers
- **Previous:** [03_optimization_goals.ipynb](03_optimization_goals.ipynb) - Goal-based configuration

---

## Before You Begin

**Required knowledge:**
- Familiarity with WorkflowTier (STANDARD, CHUNKED, STREAMING)
- Understanding of OptimizationGoal (FAST, ROBUST, QUALITY)
- Basic knowledge of memory constraints in curve fitting

**Required software:**
- NLSQ >= 0.3.4
- Python >= 3.12

---

## Why This Matters

Manual workflow selection requires understanding your dataset size, available memory,
and optimization goals. The `WorkflowSelector` encapsulates this logic, providing:

- **Automatic tier selection** based on a decision matrix
- **Memory-aware configuration** that adapts to your system
- **Goal-driven optimization** with appropriate tolerances
- **Runtime adaptation** - memory is re-evaluated on each call

Understanding how `WorkflowSelector` works helps you:
- Debug unexpected tier selections
- Override when automatic selection isn't optimal
- Configure memory limits for reproducible behavior

---

## Quick Start (30 seconds)

In [ ]:
# Configure matplotlib for inline plotting (MUST come before imports)
%matplotlib inline

In [ ]:
from nlsq.workflow import auto_select_workflow, OptimizationGoal

# Automatic workflow selection for a 5M point dataset
config = auto_select_workflow(n_points=5_000_000, n_params=5)
print(f"Selected config: {type(config).__name__}")

---

## Setup

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

from nlsq.workflow import (
    WorkflowSelector,
    WorkflowTier,
    OptimizationGoal,
    DatasetSizeTier,
    MemoryTier,
    auto_select_workflow,
    calculate_adaptive_tolerances,
)
from nlsq.large_dataset import MemoryEstimator, GPUMemoryEstimator, get_memory_tier

# Set random seed for reproducibility
np.random.seed(42)

---

## Tutorial Content

### Section 1: The Selection Matrix

The `WorkflowSelector` uses a decision matrix that maps (dataset_size, memory_tier, goal)
to (workflow_tier, enable_multistart, enable_checkpoints).

In [ ]:
# Display the selection matrix
print("WorkflowSelector Decision Matrix")
print("=" * 80)
print()
print("Dataset Size    | Low (<16GB) | Medium (16-64GB) | High (64-128GB) | Very High (>128GB)")
print("-" * 80)
print("Small (<10K)    | standard    | standard         | standard        | standard+quality")
print("Medium (10K-1M) | chunked     | standard         | standard+ms     | standard+ms")
print("Large (1M-10M)  | streaming   | chunked          | chunked+ms      | chunked+ms")
print("Huge (10M-100M) | stream+ckpt | streaming        | chunked         | chunked+ms")
print("Massive (>100M) | stream+ckpt | streaming+ckpt   | streaming       | streaming+ms")
print()
print("Legend:")
print("  ms = multi-start enabled")
print("  ckpt = checkpointing enabled")

### Section 2: Dataset Size Tiers

`DatasetSizeTier` classifies datasets and recommends tolerances.

In [ ]:
# Display all dataset size tiers
print("DatasetSizeTier Classification")
print("=" * 60)
print(f"{'Tier':<12s} | {'Max Points':<15s} | {'Tolerance':<12s}")
print("-" * 60)

for tier in DatasetSizeTier:
    max_pts = tier.max_points
    tol = tier.tolerance
    
    if max_pts == float("inf"):
        max_str = "unlimited"
    elif max_pts >= 1_000_000:
        max_str = f"{max_pts/1_000_000:.0f}M"
    elif max_pts >= 1_000:
        max_str = f"{max_pts/1_000:.0f}K"
    else:
        max_str = str(max_pts)
    
    print(f"{tier.name:<12s} | {max_str:<15s} | {tol:.0e}")

In [ ]:
# Demonstrate tier classification
test_sizes = [500, 5_000, 50_000, 500_000, 5_000_000, 50_000_000, 500_000_000]

print("\nAutomatic Size Classification")
print("-" * 40)

for n_points in test_sizes:
    tier = DatasetSizeTier.from_n_points(n_points)
    
    if n_points >= 1_000_000:
        size_str = f"{n_points/1_000_000:.0f}M"
    elif n_points >= 1_000:
        size_str = f"{n_points/1_000:.0f}K"
    else:
        size_str = str(n_points)
    
    print(f"{size_str:>8s} points -> {tier.name}")

### Section 3: Memory Tiers

`MemoryTier` classifies available system memory to inform tier selection.

In [ ]:
# Display memory tier thresholds
print("MemoryTier Classification")
print("=" * 60)
print(f"{'Tier':<12s} | {'Max Memory':<12s} | {'Description'}")
print("-" * 60)

for tier in MemoryTier:
    max_mem = tier.max_memory_gb
    desc = tier.description
    
    if max_mem == float("inf"):
        max_str = "unlimited"
    else:
        max_str = f"{max_mem:.0f} GB"
    
    print(f"{tier.name:<12s} | {max_str:<12s} | {desc}")

In [ ]:
# Check current system memory
cpu_memory = MemoryEstimator.get_available_memory_gb()
total_memory = MemoryEstimator.get_total_available_memory_gb()
current_tier = get_memory_tier(total_memory)

print("Current System Memory")
print("-" * 40)
print(f"CPU available:   {cpu_memory:.1f} GB")

# Check for GPU memory
gpu_estimator = GPUMemoryEstimator()
if gpu_estimator.has_gpu():
    gpu_memory = gpu_estimator.get_available_gpu_memory_gb()
    print(f"GPU available:   {gpu_memory:.1f} GB")
else:
    print("GPU available:   N/A (CPU only)")

print(f"Total available: {total_memory:.1f} GB")
print(f"Memory tier:     {current_tier.name}")

In [ ]:
# Demonstrate memory tier classification
test_memories = [8.0, 24.0, 48.0, 96.0, 256.0]

print("\nMemory Classification Examples")
print("-" * 40)

for mem_gb in test_memories:
    tier = MemoryTier.from_available_memory_gb(mem_gb)
    print(f"{mem_gb:>6.0f} GB -> {tier.name}")

### Section 4: WorkflowSelector in Action

Let's see how `WorkflowSelector` makes decisions for various scenarios.

In [ ]:
# Create a selector with current system memory
selector = WorkflowSelector()

print(f"WorkflowSelector initialized (using system memory: {total_memory:.1f} GB)")

In [ ]:
# Test different dataset sizes
test_scenarios = [
    (1_000, 3, None),
    (100_000, 5, None),
    (1_000_000, 5, OptimizationGoal.FAST),
    (1_000_000, 5, OptimizationGoal.QUALITY),
    (10_000_000, 5, OptimizationGoal.ROBUST),
    (100_000_000, 5, OptimizationGoal.MEMORY_EFFICIENT),
]

print("WorkflowSelector Decisions")
print("=" * 80)
print(f"{'n_points':<12s} | {'n_params':<8s} | {'Goal':<18s} | {'Config Type'}")
print("-" * 80)

for n_points, n_params, goal in test_scenarios:
    config = selector.select(n_points, n_params, goal)
    config_type = type(config).__name__
    
    if n_points >= 1_000_000:
        size_str = f"{n_points/1_000_000:.0f}M"
    elif n_points >= 1_000:
        size_str = f"{n_points/1_000:.0f}K"
    else:
        size_str = str(n_points)
    
    goal_str = goal.name if goal else "None (ROBUST)"
    
    print(f"{size_str:<12s} | {n_params:<8d} | {goal_str:<18s} | {config_type}")

In [ ]:
# Demonstrate with fixed memory limits
print("\nSelection with Different Memory Limits")
print("=" * 70)

memory_limits = [8.0, 32.0, 96.0, 256.0]  # GB
n_points = 5_000_000  # 5M points
n_params = 5

for mem_limit in memory_limits:
    selector_fixed = WorkflowSelector(memory_limit_gb=mem_limit)
    config = selector_fixed.select(n_points, n_params)
    config_type = type(config).__name__
    mem_tier = MemoryTier.from_available_memory_gb(mem_limit)
    
    print(f"Memory: {mem_limit:>6.0f} GB ({mem_tier.name:<10s}) -> {config_type}")

### Section 5: The auto_select_workflow() Convenience Function

For simple use cases, `auto_select_workflow()` provides a clean interface.

In [ ]:
# Basic usage
config1 = auto_select_workflow(n_points=5_000, n_params=5)
print(f"5K points: {type(config1).__name__}")

# With goal specification
config2 = auto_select_workflow(
    n_points=5_000_000,
    n_params=5,
    goal=OptimizationGoal.QUALITY,
)
print(f"5M points + QUALITY: {type(config2).__name__}")

# With memory override
config3 = auto_select_workflow(
    n_points=5_000_000,
    n_params=5,
    memory_limit_gb=8.0,
)
print(f"5M points + 8GB limit: {type(config3).__name__}")

### Section 6: Adaptive Tolerances

The selector also calculates adaptive tolerances based on dataset size and goal.

In [ ]:
# Demonstrate adaptive tolerance calculation
print("Adaptive Tolerance Calculation")
print("=" * 60)

test_configs = [
    (1_000, None),
    (1_000_000, None),
    (1_000_000, OptimizationGoal.FAST),
    (1_000_000, OptimizationGoal.QUALITY),
    (100_000_000, OptimizationGoal.ROBUST),
]

print(f"{'n_points':<12s} | {'Goal':<12s} | {'gtol':<12s} | {'ftol':<12s}")
print("-" * 60)

for n_points, goal in test_configs:
    tolerances = calculate_adaptive_tolerances(n_points, goal)
    
    if n_points >= 1_000_000:
        size_str = f"{n_points/1_000_000:.0f}M"
    else:
        size_str = f"{n_points/1_000:.0f}K"
    
    goal_str = goal.name if goal else "None"
    
    print(f"{size_str:<12s} | {goal_str:<12s} | {tolerances['gtol']:.0e} | {tolerances['ftol']:.0e}")

### Section 7: Visualization of the Selection Algorithm

Let's visualize the complete selection process.

In [ ]:
# Create a comprehensive visualization of the selection algorithm
fig, ax = plt.subplots(figsize=(16, 12))
ax.set_xlim(0, 16)
ax.set_ylim(0, 14)
ax.axis('off')

# Title
ax.text(8, 13.5, "WorkflowSelector Decision Algorithm", ha='center', fontsize=18, fontweight='bold')

# Step 1: Input
ax.add_patch(plt.Rectangle((0.5, 11.5), 3, 1.2, fill=True, facecolor='lightblue', edgecolor='black', linewidth=2))
ax.text(2, 12.1, "INPUT", ha='center', va='center', fontsize=12, fontweight='bold')
ax.text(2, 11.7, "n_points, n_params, goal", ha='center', va='center', fontsize=9)

# Arrow down
ax.annotate('', xy=(2, 10.3), xytext=(2, 11.5),
            arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Step 2: Get Memory
ax.add_patch(plt.Rectangle((0.5, 9.3), 3, 1, fill=True, facecolor='lightyellow', edgecolor='black'))
ax.text(2, 9.8, "1. Get Memory", ha='center', va='center', fontsize=10, fontweight='bold')
ax.text(2, 9.5, "get_available_memory_gb()", ha='center', va='center', fontsize=8)

# Arrow down
ax.annotate('', xy=(2, 8.1), xytext=(2, 9.3),
            arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Step 3: Classify Memory
ax.add_patch(plt.Rectangle((0.5, 7.1), 3, 1, fill=True, facecolor='lightyellow', edgecolor='black'))
ax.text(2, 7.6, "2. Classify Memory", ha='center', va='center', fontsize=10, fontweight='bold')
ax.text(2, 7.3, "MemoryTier.from_...()", ha='center', va='center', fontsize=8)

# Arrow down
ax.annotate('', xy=(2, 5.9), xytext=(2, 7.1),
            arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Step 4: Classify Dataset
ax.add_patch(plt.Rectangle((0.5, 4.9), 3, 1, fill=True, facecolor='lightyellow', edgecolor='black'))
ax.text(2, 5.4, "3. Classify Dataset", ha='center', va='center', fontsize=10, fontweight='bold')
ax.text(2, 5.1, "DatasetSizeTier.from_...()", ha='center', va='center', fontsize=8)

# Arrow down
ax.annotate('', xy=(2, 3.7), xytext=(2, 4.9),
            arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Step 5: Apply Matrix
ax.add_patch(plt.Rectangle((0.5, 2.7), 3, 1, fill=True, facecolor='lightgreen', edgecolor='black', linewidth=2))
ax.text(2, 3.2, "4. Decision Matrix", ha='center', va='center', fontsize=10, fontweight='bold')
ax.text(2, 2.9, "tier, multistart, ckpt", ha='center', va='center', fontsize=8)

# Arrow down
ax.annotate('', xy=(2, 1.5), xytext=(2, 2.7),
            arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Step 6: Output
ax.add_patch(plt.Rectangle((0.5, 0.5), 3, 1, fill=True, facecolor='lightsalmon', edgecolor='black', linewidth=2))
ax.text(2, 1, "OUTPUT", ha='center', va='center', fontsize=12, fontweight='bold')
ax.text(2, 0.7, "ConfigType", ha='center', va='center', fontsize=9)

# Memory Tier Reference (right side)
ax.add_patch(plt.Rectangle((5, 8), 4.5, 4.5, fill=True, facecolor='white', edgecolor='black'))
ax.text(7.25, 12, "MemoryTier", ha='center', fontsize=11, fontweight='bold')
ax.text(5.2, 11.3, "LOW: < 16 GB", fontsize=9)
ax.text(5.2, 10.6, "MEDIUM: 16-64 GB", fontsize=9)
ax.text(5.2, 9.9, "HIGH: 64-128 GB", fontsize=9)
ax.text(5.2, 9.2, "VERY_HIGH: > 128 GB", fontsize=9)
ax.text(5.2, 8.4, f"Current: {current_tier.name}", fontsize=9, fontweight='bold', color='blue')

# Dataset Size Reference
ax.add_patch(plt.Rectangle((10, 8), 5.5, 4.5, fill=True, facecolor='white', edgecolor='black'))
ax.text(12.75, 12, "DatasetSizeTier", ha='center', fontsize=11, fontweight='bold')
ax.text(10.2, 11.3, "TINY: < 1K (tol=1e-12)", fontsize=9)
ax.text(10.2, 10.7, "SMALL: 1K-10K (tol=1e-10)", fontsize=9)
ax.text(10.2, 10.1, "MEDIUM: 10K-100K (tol=1e-9)", fontsize=9)
ax.text(10.2, 9.5, "LARGE: 100K-1M (tol=1e-8)", fontsize=9)
ax.text(10.2, 8.9, "VERY_LARGE: 1M-10M (tol=1e-7)", fontsize=9)
ax.text(10.2, 8.3, "HUGE/MASSIVE: >10M", fontsize=9)

# Config Types Reference
ax.add_patch(plt.Rectangle((5, 0.5), 10.5, 3, fill=True, facecolor='white', edgecolor='black'))
ax.text(10.25, 3, "Output Config Types", ha='center', fontsize=11, fontweight='bold')
ax.text(5.2, 2.3, "GlobalOptimizationConfig: STANDARD + multistart", fontsize=9, color='green')
ax.text(5.2, 1.7, "LDMemoryConfig: STANDARD or CHUNKED", fontsize=9, color='orange')
ax.text(5.2, 1.1, "HybridStreamingConfig: STREAMING or STREAMING_CHECKPOINT", fontsize=9, color='red')

# Goal modifiers
ax.add_patch(plt.Rectangle((5, 4.5), 10.5, 3, fill=True, facecolor='lavender', edgecolor='black'))
ax.text(10.25, 7, "Goal Modifiers", ha='center', fontsize=11, fontweight='bold')
ax.text(5.2, 6.3, "FAST: Disable multistart, looser tolerances", fontsize=9)
ax.text(5.2, 5.7, "ROBUST/GLOBAL: Enable multistart (if memory allows)", fontsize=9)
ax.text(5.2, 5.1, "QUALITY: Enable multistart + tighter tolerances", fontsize=9)
ax.text(5.2, 4.7, "MEMORY_EFFICIENT: Force streaming/chunking", fontsize=9)

plt.tight_layout()
plt.savefig("figures/06_selection_algorithm.png", dpi=300, bbox_inches="tight")
plt.show()

---

## Key Takeaways

After completing this notebook, remember:

1. **WorkflowSelector uses a decision matrix** that maps (dataset_size, memory_tier, goal) to (tier, multistart, checkpoints).

2. **Memory is re-evaluated on each call** - no caching, ensuring accurate adaptation to current conditions.

3. **Three tier classifications:**
   - `DatasetSizeTier`: TINY, SMALL, MEDIUM, LARGE, VERY_LARGE, HUGE, MASSIVE
   - `MemoryTier`: LOW, MEDIUM, HIGH, VERY_HIGH
   - `WorkflowTier`: STANDARD, CHUNKED, STREAMING, STREAMING_CHECKPOINT

4. **Three output config types:**
   - `GlobalOptimizationConfig`: For STANDARD + multistart
   - `LDMemoryConfig`: For STANDARD or CHUNKED
   - `HybridStreamingConfig`: For STREAMING or STREAMING_CHECKPOINT

5. **Use `auto_select_workflow()` for simple cases:**
   ```python
   config = auto_select_workflow(n_points=5_000_000, n_params=5)
   ```

---

## Common Questions

**Q: Why does the same code produce different configs on different machines?**

A: Memory is detected at runtime. A machine with 32GB will get different tier selections than one with 128GB. Use `memory_limit_gb` parameter for reproducibility.

**Q: How do I force a specific tier?**

A: Either use `WorkflowConfig(tier=WorkflowTier.STREAMING)` or set a very low `memory_limit_gb` to force streaming.

**Q: What if I disagree with the automatic selection?**

A: The selector provides a sensible default. Override with explicit configuration when your domain knowledge suggests otherwise.

---

## Related Resources

**Prerequisites:**
- [02_workflow_tiers.ipynb](02_workflow_tiers.ipynb) - Tier fundamentals
- [03_optimization_goals.ipynb](03_optimization_goals.ipynb) - Goal-based configuration

**Further reading:**
- [API Documentation: workflow module](https://nlsq.readthedocs.io/api/workflow/)

---

## Glossary

**Decision Matrix:** A lookup table mapping input conditions to output configurations.

**DatasetSizeTier:** Classification of dataset size with recommended tolerances.

**MemoryTier:** Classification of available system memory (CPU + GPU).

**WorkflowSelector:** Class that implements the automatic workflow selection algorithm.

**auto_select_workflow:** Convenience function wrapping WorkflowSelector.select().

In [ ]:
# Final summary
print("Summary")
print("=" * 60)
print()
print("Key Functions:")
print("  auto_select_workflow(n_points, n_params, goal)")
print("  WorkflowSelector().select(n_points, n_params, goal)")
print("  DatasetSizeTier.from_n_points(n_points)")
print("  MemoryTier.from_available_memory_gb(memory_gb)")
print("  MemoryEstimator.get_total_available_memory_gb()")
print("  get_memory_tier(memory_gb)")
print()
print(f"Current System:")
print(f"  Total memory: {total_memory:.1f} GB")
print(f"  Memory tier: {current_tier.name}")